## Setup
1. In `cd NEMO`
1. Run YAP API server `./yap api` (if you specify a port, change it in `config.py`)
1. `pip install -r requirements.txt`
1. Run NEMO API server `uvicorn api_main:app --reload --port 8090`
1. Have a look at the swagger OpenAPI documentation by opening http://localhost:8090/docs in your browser

In [1]:
import requests

In [2]:
requests.get('http://localhost:8090/').json()

{'message': 'Please specify command in URL path.',
 'available_commands': ['run_ner_model',
  'multi_align_hybrid',
  'multi_to_single',
  'morph_yap',
  'morph_hybrid',
  'morph_hybrid_align_tokens']}

In [3]:
texts = [
    'מלך השערים במונדיאל 2006 היה מירוסלב קלוזה.',
    "עשרות אנשים מגיעים מתאילנד לישראל.\nתופעה זו התבררה אתמול בוועדת העבודה והרווחה של הכנסת."
]

## Run NER Model

In [4]:
%%time 
payload = { 'sentences': texts[1],
            'model_name': 'token-multi', # / 'token-single' / 'morph' 
            #'tokenized': True,
          }

res = requests.get('http://localhost:8090/run_ner_model', params=payload).json()
list(zip(res[0]['tokenized_text'], res[0]['ncrf_preds'])), list(zip(res[1]['tokenized_text'], res[1]['ncrf_preds']))

CPU times: user 4.83 ms, sys: 2.81 ms, total: 7.64 ms
Wall time: 96.1 ms


([('עשרות', 'O'),
  ('אנשים', 'O'),
  ('מגיעים', 'O'),
  ('מתאילנד', 'O^S-GPE'),
  ('לישראל', 'O^S-GPE'),
  ('.', 'O')],
 [('תופעה', 'O'),
  ('זו', 'O'),
  ('התבררה', 'O'),
  ('אתמול', 'O'),
  ('בוועדת', 'O^B-ORG'),
  ('העבודה', 'I-ORG^I-ORG'),
  ('והרווחה', 'I-ORG^I-ORG^E-ORG'),
  ('של', 'O'),
  ('הכנסת', 'B-ORG^E-ORG'),
  ('.', 'O')])

### multi_to_single (get token-single predictions using a token-multi model)

In [5]:
%%time
payload = { 'sentences': texts[1],
            #'multi_model_name': 'token-multi',
            #'tokenized': True,
          }

res = requests.get('http://localhost:8090/multi_to_single', params=payload).json()
res

CPU times: user 6.06 ms, sys: 1 ms, total: 7.07 ms
Wall time: 69.5 ms


[{'tokenized_text': ['עשרות', 'אנשים', 'מגיעים', 'מתאילנד', 'לישראל', '.'],
  'multi_ncrf_preds': ['O', 'O', 'O', 'O^S-GPE', 'O^S-GPE', 'O'],
  'multi_ncrf_preds_align_single': ['O', 'O', 'O', 'S-GPE', 'S-GPE', 'O']},
 {'tokenized_text': ['תופעה',
   'זו',
   'התבררה',
   'אתמול',
   'בוועדת',
   'העבודה',
   'והרווחה',
   'של',
   'הכנסת',
   '.'],
  'multi_ncrf_preds': ['O',
   'O',
   'O',
   'O',
   'O^B-ORG',
   'I-ORG^I-ORG',
   'I-ORG^I-ORG^E-ORG',
   'O',
   'B-ORG^E-ORG',
   'O'],
  'multi_ncrf_preds_align_single': ['O',
   'O',
   'O',
   'O',
   'B-ORG',
   'I-ORG',
   'E-ORG',
   'O',
   'S-ORG',
   'O']}]

## MD + NER

### multi_align_hybrid

In [6]:
%%time
payload = { 'sentences': texts[1],
            #'multi_model_name': 'token-multi',
            #'tokenized': True,
          }

res = requests.get('http://localhost:8090/multi_align_hybrid', params=payload).json()
res

CPU times: user 5.8 ms, sys: 1.43 ms, total: 7.23 ms
Wall time: 247 ms


[{'tokenized_text': ['עשרות', 'אנשים', 'מגיעים', 'מתאילנד', 'לישראל', '.'],
  'ma_lattice': '0\t1\tעשרות\tעשר\tCDT\tCDT\tgen=F|num=P\t1\n0\t1\tעשרות\tעשר\tCD\tCD\tgen=F|num=P\t1\n1\t2\tאנשים\tהנשים\tVB\tVB\tgen=F|gen=M|num=S|per=1|tense=FUTURE\t2\n1\t2\tאנשים\tאיש\tNN\tNN\tgen=M|num=P\t2\n2\t3\tמגיעים\tהגיע\tVB\tVB\tgen=M|num=P|per=A|tense=BEINONI\t3\n2\t3\tמגיעים\tהגיע\tBN\tBN\tgen=M|num=P|per=A\t3\n3\t4\tמ\tמ\tPREPOSITION\tPREPOSITION\t_\t4\n3\t5\tמתאילנד\tמתאילנד\tNNP\tNNP\tgen=M|num=S\t4\n3\t5\tמתאילנד\tמתאילנד\tNN\tNN\tgen=M|num=P|num=S\t4\n3\t5\tמתאילנד\tמתאילנד\tNN\tNN\tgen=M|num=S\t4\n3\t5\tמתאילנד\tמתאילנד\tNNP\tNNP\tgen=F|num=S\t4\n3\t5\tמתאילנד\tמתאילנד\tNNP\tNNP\tgen=F|gen=M|num=S\t4\n3\t5\tמתאילנד\tמתאילנד\tNNP\tNNP\t_\t4\n3\t5\tמתאילנד\tמתאילנד\tNN\tNN\tgen=M|num=P\t4\n3\t5\tמתאילנד\tמתאילנד\tNN\tNN\tgen=F|num=S\t4\n3\t5\tמתאילנד\tמתאילנד\tNN\tNN\tgen=F|num=P\t4\n4\t5\tתאילנד\tתאילנד\tNNP\tNNP\tgen=F|num=S\t4\n5\t6\tל\tל\tPREPOSITION\tPREPOSITION\t_\t5\n5\t8\tלישראל\tלישר

In [7]:
print(res[0]['md_lattice'])

0	1	עשרות	עשר	CDT	CDT	gen=F|num=P	1
1	2	אנשים	איש	NN	NN	gen=M|num=P	2
2	3	מגיעים	הגיע	BN	BN	gen=M|num=P|per=A	3
3	4	מ	מ	PREPOSITION	PREPOSITION	_	4
4	5	תאילנד	תאילנד	NNP	NNP	gen=F|num=S	4
5	6	ל	ל	PREPOSITION	PREPOSITION	_	5
6	7	ישראל	ישראל	NNP	NNP	gen=F|num=S	5
7	8	.	_	yyDOT	yyDOT	_	6


### morph_yap

In [8]:
%%time
payload = { 'sentences': texts[1],
            #'morph_model_name': 'morph',
            #'tokenized': True,
          }

res = requests.get('http://localhost:8090/morph_yap', params=payload).json()
res

CPU times: user 6.68 ms, sys: 2.25 ms, total: 8.92 ms
Wall time: 335 ms


[{'tokenized_text': ['עשרות', 'אנשים', 'מגיעים', 'מתאילנד', 'לישראל', '.'],
  'ma_lattice': '0\t1\tעשרות\tעשר\tCDT\tCDT\tgen=F|num=P\t1\n0\t1\tעשרות\tעשר\tCD\tCD\tgen=F|num=P\t1\n1\t2\tאנשים\tהנשים\tVB\tVB\tgen=F|gen=M|num=S|per=1|tense=FUTURE\t2\n1\t2\tאנשים\tאיש\tNN\tNN\tgen=M|num=P\t2\n2\t3\tמגיעים\tהגיע\tVB\tVB\tgen=M|num=P|per=A|tense=BEINONI\t3\n2\t3\tמגיעים\tהגיע\tBN\tBN\tgen=M|num=P|per=A\t3\n3\t4\tמ\tמ\tPREPOSITION\tPREPOSITION\t_\t4\n3\t5\tמתאילנד\tמתאילנד\tNNP\tNNP\tgen=M|num=S\t4\n3\t5\tמתאילנד\tמתאילנד\tNN\tNN\tgen=M|num=P|num=S\t4\n3\t5\tמתאילנד\tמתאילנד\tNN\tNN\tgen=M|num=S\t4\n3\t5\tמתאילנד\tמתאילנד\tNNP\tNNP\tgen=F|num=S\t4\n3\t5\tמתאילנד\tמתאילנד\tNNP\tNNP\tgen=F|gen=M|num=S\t4\n3\t5\tמתאילנד\tמתאילנד\tNNP\tNNP\t_\t4\n3\t5\tמתאילנד\tמתאילנד\tNN\tNN\tgen=M|num=P\t4\n3\t5\tמתאילנד\tמתאילנד\tNN\tNN\tgen=F|num=S\t4\n3\t5\tמתאילנד\tמתאילנד\tNN\tNN\tgen=F|num=P\t4\n4\t5\tתאילנד\tתאילנד\tNNP\tNNP\tgen=F|num=S\t4\n5\t6\tל\tל\tPREPOSITION\tPREPOSITION\t_\t5\n5\t8\tלישראל\tלישר

### morph_hybrid
The following models got the best results in our experiments.


In [9]:
%%time
payload = { 'sentences': texts[1],
            #'multi_model_name': 'token-multi',
            #'morph_model_name': 'morph',
            #'tokenized': False,
            # 'align_tokens': False,
          }

res = requests.get('http://localhost:8090/morph_hybrid', params=payload).json()
res

CPU times: user 8.16 ms, sys: 1.79 ms, total: 9.95 ms
Wall time: 311 ms


[{'tokenized_text': ['עשרות', 'אנשים', 'מגיעים', 'מתאילנד', 'לישראל', '.'],
  'ma_lattice': '0\t1\tעשרות\tעשר\tCDT\tCDT\tgen=F|num=P\t1\n0\t1\tעשרות\tעשר\tCD\tCD\tgen=F|num=P\t1\n1\t2\tאנשים\tהנשים\tVB\tVB\tgen=F|gen=M|num=S|per=1|tense=FUTURE\t2\n1\t2\tאנשים\tאיש\tNN\tNN\tgen=M|num=P\t2\n2\t3\tמגיעים\tהגיע\tVB\tVB\tgen=M|num=P|per=A|tense=BEINONI\t3\n2\t3\tמגיעים\tהגיע\tBN\tBN\tgen=M|num=P|per=A\t3\n3\t4\tמ\tמ\tPREPOSITION\tPREPOSITION\t_\t4\n3\t5\tמתאילנד\tמתאילנד\tNNP\tNNP\tgen=M|num=S\t4\n3\t5\tמתאילנד\tמתאילנד\tNN\tNN\tgen=M|num=P|num=S\t4\n3\t5\tמתאילנד\tמתאילנד\tNN\tNN\tgen=M|num=S\t4\n3\t5\tמתאילנד\tמתאילנד\tNNP\tNNP\tgen=F|num=S\t4\n3\t5\tמתאילנד\tמתאילנד\tNNP\tNNP\tgen=F|gen=M|num=S\t4\n3\t5\tמתאילנד\tמתאילנד\tNNP\tNNP\t_\t4\n3\t5\tמתאילנד\tמתאילנד\tNN\tNN\tgen=M|num=P\t4\n3\t5\tמתאילנד\tמתאילנד\tNN\tNN\tgen=F|num=S\t4\n3\t5\tמתאילנד\tמתאילנד\tNN\tNN\tgen=F|num=P\t4\n4\t5\tתאילנד\tתאילנד\tNNP\tNNP\tgen=F|num=S\t4\n5\t6\tל\tל\tPREPOSITION\tPREPOSITION\t_\t5\n5\t8\tלישראל\tלישר

### morph_hybrid_align_tokens

In [10]:
%%time
payload = { 'sentences': texts[1],
            #'tokenized': False,
            #'align_tokens': False,
          }

res = requests.get('http://localhost:8090/morph_hybrid_align_tokens', params=payload).json()
res

CPU times: user 9.22 ms, sys: 2.05 ms, total: 11.3 ms
Wall time: 357 ms


[{'tokenized_text': ['עשרות', 'אנשים', 'מגיעים', 'מתאילנד', 'לישראל', '.'],
  'ma_lattice': '0\t1\tעשרות\tעשר\tCDT\tCDT\tgen=F|num=P\t1\n0\t1\tעשרות\tעשר\tCD\tCD\tgen=F|num=P\t1\n1\t2\tאנשים\tהנשים\tVB\tVB\tgen=F|gen=M|num=S|per=1|tense=FUTURE\t2\n1\t2\tאנשים\tאיש\tNN\tNN\tgen=M|num=P\t2\n2\t3\tמגיעים\tהגיע\tVB\tVB\tgen=M|num=P|per=A|tense=BEINONI\t3\n2\t3\tמגיעים\tהגיע\tBN\tBN\tgen=M|num=P|per=A\t3\n3\t4\tמ\tמ\tPREPOSITION\tPREPOSITION\t_\t4\n3\t5\tמתאילנד\tמתאילנד\tNNP\tNNP\tgen=M|num=S\t4\n3\t5\tמתאילנד\tמתאילנד\tNN\tNN\tgen=M|num=P|num=S\t4\n3\t5\tמתאילנד\tמתאילנד\tNN\tNN\tgen=M|num=S\t4\n3\t5\tמתאילנד\tמתאילנד\tNNP\tNNP\tgen=F|num=S\t4\n3\t5\tמתאילנד\tמתאילנד\tNNP\tNNP\tgen=F|gen=M|num=S\t4\n3\t5\tמתאילנד\tמתאילנד\tNNP\tNNP\t_\t4\n3\t5\tמתאילנד\tמתאילנד\tNN\tNN\tgen=M|num=P\t4\n3\t5\tמתאילנד\tמתאילנד\tNN\tNN\tgen=F|num=S\t4\n3\t5\tמתאילנד\tמתאילנד\tNN\tNN\tgen=F|num=P\t4\n4\t5\tתאילנד\tתאילנד\tNNP\tNNP\tgen=F|num=S\t4\n5\t6\tל\tל\tPREPOSITION\tPREPOSITION\t_\t5\n5\t8\tלישראל\tלישר

## Working with sequence labels
`iobes` can be used to parse the predictions (`pip install iobes`)

In [15]:
from notebook_utils import *

In [16]:
payload = { 'sentences': '\n'.join(texts),}

res = requests.get('http://localhost:8090/morph_hybrid_align_tokens', params=payload).json()

In [17]:
ents_hyb = get_ents(res, 'morph_ncrf_preds', 'morph_forms')
ents_hyb

,sent_id,text,cat
0,1,מירוסלב קלוזה,PER
1,2,תאילנד,GPE
2,2,ישראל,GPE
3,3,וועדת ה עבודה ו ה רווחה,ORG
4,3,ה כנסת,ORG


In [18]:
ents_hyb = get_ents(res, 'multi_ncrf_preds_align_morph', 'morph_forms')
ents_hyb

,sent_id,text,cat
0,1,מירוסלב קלוזה,PER
1,2,תאילנד,GPE
2,2,ישראל,GPE
3,3,וועדת ה עבודה ו ה רווחה,ORG
4,3,ה כנסת,ORG


In [19]:
ents_hyb = get_ents(res, 'multi_ncrf_preds_align_single', 'tokenized_text')
ents_hyb

,sent_id,text,cat
0,1,מירוסלב קלוזה,PER
1,2,מתאילנד,GPE
2,2,לישראל,GPE
3,3,בוועדת העבודה והרווחה,ORG
4,3,הכנסת,ORG


In [20]:
spans = get_spans(res, 'morph_ncrf_preds', 'morph_forms')
e = EntityRenderer()

In [21]:
from IPython.core.display import display, HTML
display(HTML(e.render(spans)))
